# Three ways to connect to milvus from a notebook

There are 5 different ways to start up a Milvus server.
1. 🥳 Milvus Local Python server is coming soon!  
   ⛔️ Only meant for demos and local testing.

💡 **For production purposes**, it is recommended to use Milvus local docker, clusters, or fully-managed Milvus on Zilliz Cloud. <br>

2. [Milvus standalone docker](https://milvus.io/docs/install_standalone-docker.md) requires local docker installed and running.
3. [Ziliz Cloud free tier](#zilliz_free)
4. <a href="https://milvus.io/docs/install_cluster-milvusoperator.md">Milvus clusters</a> requires a K8s cluster up and running.
5. <a href="https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.milvus.Milvus.html">LangChain Milvus</a> and other 3rd party adapters.

Once your milvus server or cluster is up and running, you're all set to make API calls.  I'll demonstrate how to do this using the [Python SDK](https://github.com/milvus-io/pymilvus/blob/master/pymilvus/milvus_client/milvus_client.py).  You can also check out a [practical example](https://github.com/milvus-io/pymilvus/blob/bac31951d5c5a9dacb6632e535e3c4d284726390/examples/hello_milvus_simple.py).  

### Notebook contents for 2 ways to connect
1. [Local milvus on docker demo](#milvus_docker)
2. [Zilliz cloud free tier](#zilliz_free)
3. Milvus clusters on K8 - demo coming soon
4. 3rd party adapters - demo coming soon

## 1. Milvus standalone server running on Docker <a class="anchor" id="milvus_docker"></a>

This section uses [Milvus standalone](https://milvus.io/docs/configure-docker.md) on Docker on your laptop. <br>
>⛔️ Make sure you pip install the correct version of milvus, pymilvus, and server yml file.  **Versions (major and minor) should all match**.

1. [Install Docker](https://docs.docker.com/get-docker/)
2. Start your Docker Desktop
3. Download the latest [docker-compose.yml](https://milvus.io/docs/install_standalone-docker.md#Download-the-YAML-file) (or run the wget command in the next cell)
4. From you terminal:  
   - activate your python environment
   - cd into directory where you saved the .yml file (usualy same dir as this notebook)
   - docker compose up -d
   - verify (either in terminal or on Docker Desktop) the containers are running
5. Import milvus
6. Connect to the local milvus server (see notebook code below)

In [6]:
# !pip install pymilvus
import pymilvus, time
from pymilvus import (connections, MilvusClient, utility)
print(f"Pymilvus: {pymilvus.__version__}")

####################################################################################################
# Connect to local server running in Docker container.
# Download the latest .yaml file
# See https://milvus.io/docs/install_standalone-docker.md

# Above did not work for me!? I downloaded the file manually from milvus github:
# !wget https://github.com/milvus-io/milvus/releases/download/v2.4.0-rc.1/milvus-standalone-docker-compose.yml -O docker-compose.yml
####################################################################################################
# # Stop local milvus.
# !docker compose down

# Start Docker - see terminal directions above.
!docker compose up -d

# Verify which local port the Milvus server is listening on
!docker ps -a #19530/tcp

# Add wait to avoid error message from trying to connect too soon.
time.sleep(30)

# Connect to the local server.
connection = connections.connect(
  alias="default", 
  host='localhost', # or '0.0.0.0' or 'localhost'
  port='19530'
)

# Get server version.
print(utility.get_server_version())

# Use no-schema Milvus client uses flexible json key:value format.
mc = MilvusClient(connections=connection)

Pymilvus: 2.4.0
[+] Running 0/0
 ⠋ Network milvus               Created                                    0.1s 
 ⠋ Container milvus-etcd        Created                                    0.1s 
 ⠋ Container milvus-minio       Created                                    0.1s 
 ⠋ Container milvus-standalone  Creating                                   0.0s 
[+] Running 0/4
 ⠙ Network milvus               Created                                    0.2s 
 ⠙ Container milvus-etcd        Starting                                   0.2s 
 ⠙ Container milvus-minio       Starting                                   0.2s 
 ⠙ Container milvus-standalone  Created                                    0.1s 
[+] Running 0/4
 ⠹ Network milvus               Created                                    0.3s 
 ⠹ Container milvus-etcd        Starting                                   0.3s 
 ⠹ Container milvus-minio       Starting                                   0.3s 
 ⠹ Container milvus-standalone  Created      

**Error message if try to connect before docker up.**

MilvusException                           Traceback (most recent call last)
    145 except grpc.FutureTimeoutError as e:
--> 146     raise MilvusException(
    147         code=Status.CONNECT_FAILED,
    148         message=f"Fail connecting to server on {self._address}, illegal connection params or server unavailable",
    149     ) from e
    150 except Exception as e:
    151     raise e from e

MilvusException: <MilvusException: (code=2, message=Fail connecting to server on localhost:19530, illegal connection params or server unavailable)>

In [7]:
# Check if a collection already exists.
collection_name = "movies"
has = utility.has_collection(collection_name)
print(f"Collection '{collection_name}' exists in Milvus?")
print(f"Answer --> {has}!")
if has:
    drop_result = utility.drop_collection(collection_name)
    print(f"Successfully dropped {collection_name}")

Collection 'movies' exists in Milvus?
Answer --> False!


In [8]:
# Stop local milvus.
!docker compose down

[+] Running 0/0
 ⠋ Container milvus-standalone  Stopping                                   0.1s 
[+] Running 0/1
 ⠙ Container milvus-standalone  Stopping                                   0.2s 
[+] Running 0/1
 ⠹ Container milvus-standalone  Stopping                                   0.3s 
[+] Running 0/1
 ⠸ Container milvus-standalone  Stopping                                   0.4s 
[+] Running 0/1
 ⠼ Container milvus-standalone  Stopping                                   0.5s 
[+] Running 0/1
 ⠴ Container milvus-standalone  Stopping                                   0.6s 
[+] Running 0/1
 ⠦ Container milvus-standalone  Stopping                                   0.7s 
[+] Running 0/1
 ⠧ Container milvus-standalone  Stopping                                   0.8s 
[+] Running 0/1
 ⠇ Container milvus-standalone  Stopping                                   0.9s 
[+] Running 0/1
 ⠏ Container milvus-standalone  Stopping                                   1.0s 
[+] Running 0/1
 ⠋ Container m

## 2. Zilliz free tier  <a class="anchor" id="zilliz_free"></a>

This section uses [Zilliz](https://zilliz.com), free tier.  If you have not already, sign up for a [free trial](https://cloud.zilliz.com/signup).  

If you already have a Zilliz account and want to use free tier, just be sure to select "Starter" option when you [create your cluster](https://docs.zilliz.com/docs/create-cluster).  ❤️‍🔥 **In other words, everybody gets free tier!!**  
- One free tier cluster per account.
- Per free tier cluster, up to two collections at a time. (Think of a collection like a database table. Each collection has an index, schema, and consistency-level).
- Each free tier collection can support up to 1M vectors (Think of this like rows in a database table).

If you require additional clusters, we recommend subscribing to either our Standard or Enterprise plan.

### 👩 Set up instructions for Zilliz 

1. From [cloud.zilliz.com](cloud.zilliz.com), click **"+ Create Cluster"**
2. Select <i>**Starter**</i> option for the cluster and click **"Next: Create Collection"**
   <div>
   <img src="../images/zilliz_cluster_choose.png" width="60%"/>
   </div>

1. Name your collection with a <i>**Collection Name**</i> and click **"Create Collection and Cluster"**.
2. From the Clusters page, 
   - copy the cluster uri and save somewhere locally.
   - copy your cluster API KEY.  Keep this private! 
     <div>
     <img src="../images/zilliz_cluster_uri_token.png" width="80%"/>
     </div>

3. Add the API KEY to your environment variables.  See this [article for instructions](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety) how in either Windows or Mac/Linux environment.
4. In Jupyter, you'll also need .env file (in same dir as notebooks) containing lines like this:
   - ZILLIZ_API_KEY=value
5. In your code, connect to your Zilliz cluster, see code example below.

In [2]:
import os
from pymilvus import (connections, MilvusClient, utility)
TOKEN = os.getenv("ZILLIZ_API_KEY")

# Connect to Zilliz cloud using endpoint URI and API key TOKEN.
# TODO change this.
CLUSTER_ENDPOINT="https://in03-xxxx.api.gcp-us-west1.zillizcloud.com:443"
CLUSTER_ENDPOINT="https://in03-48a5b11fae525c9.api.gcp-us-west1.zillizcloud.com:443"
connections.connect(
  alias='default',
  #  Public endpoint obtained from Zilliz Cloud
  uri=CLUSTER_ENDPOINT,
  # API key or a colon-separated cluster username and password
  token=TOKEN,
)

# Check if the server is ready and get colleciton name.
print(f"Type of server: {utility.get_server_version()}")

# Use no-schema Milvus client uses flexible json key:value format.
# https://milvus.io/docs/using_milvusclient.md
mc = MilvusClient(
    uri=CLUSTER_ENDPOINT,
    # API key or a colon-separated cluster username and password
    token=TOKEN)

Type of server: Zilliz Cloud Vector Database(Compatible with Milvus 2.3)
